## Imports & Environment Variables

In [ ]:
import os
from groq import Groq
from dotenv import load_dotenv

load_dotenv()

GROQ_API_KEY = "INSERT GROQ API KEY HERE"

In [ ]:
#function to invoke groq response
def groq_invoke(prompt):
    
    client = Groq(
        api_key=GROQ_API_KEY,
    )
    
    chat_completion = client.chat.completions.create(
        messages = [
            {
                "role": "user",
                "content": prompt
            }
        ],
        model = "llama3-8b-8192"
    )
    
    return chat_completion.choices[0].message.content